# Serum proteomic pattern diagnostics

Dataset from the FDA-NCI Clinical Proteomics Program Databank [1,2].

Each column of `A` represents measurements taken from a patient. There are 216 columns representing 216 patients, out of which 121 are ovarian cancer patients and 95 are normal patients.

Each row represents the ion intensity level at a specific mass-charge value indicated in MZ. There are 2000 mass-charge values, and each row represents the ion-intensity levels of the patients at that particular mass-charge value.

The variable `grp` holds the index information as to which of these samples represent cancer patients and which ones represent normal patients.

- [1] T.P. Conrads, et al., ["High-resolution serum proteomic features for ovarian detection"](https://doi.org/10.1677/erc.0.0110163), Endocrine-Related Cancer, 11, 2004, pp. 163-178.
- [2] E.F. Petricoin, et al., ["Use of proteomic patterns in serum to identify ovarian cancer"](<https://doi.org/10.1016/s0140-6736(02)07746-2>), Lancet, 359(9306), 2002, pp. 572-577.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

ovariancancer_obs_path = "./ovariancancer_obs.csv"
ovariancancer_grp_path = "./ovariancancer_grp.csv"

A = np.genfromtxt(ovariancancer_obs_path, delimiter=",").transpose()
with open(ovariancancer_grp_path, "r") as fp:
    grp = np.array(fp.read().split("\n"))
grp = grp[grp != ""]

How many patients? How many features?


In [ ]:
n_features = A.shape[0]
n_patients = A.shape[1]
print(f"{n_features} features")
print(f"{n_patients} patients")

How many patiens with and without cancer are there in the dataset?


In [ ]:
print(f"{np.sum(grp == 'Cancer')} Cancer")
print(f"{np.sum(grp == 'Normal')} Normal")
print(f"Total: {grp.size}")
is_sane = np.where(grp == "Normal", 1, 0)

Choose two proteins (between 1 and 2000), and make a scatterplot of the corresponding values plotting with different colors the data corresponding to patients with and without cancer. Can you easily distinguish between patients with/without cancer?


In [ ]:
protein_x = 0
protein_y = 1

plt.scatter(
    A[protein_x, grp == "Cancer"], A[protein_y, grp == "Cancer"], label="Cancer"
)
plt.scatter(
    A[protein_x, grp == "Normal"], A[protein_y, grp == "Normal"], label="Normal"
)
plt.legend()
plt.show()

Repeat the previous point, this time by considering three proteins.


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

protein_x = 1000
protein_y = 2000
protein_z = 3000

for label in ("Cancer", "Normal"):
    ax.scatter(
        A[protein_x, grp == label],
        A[protein_y, grp == label],
        A[protein_z, grp == label],
        label=label,
        marker="x",
        s=50,
    )
plt.legend()
ax.view_init(25, 20)

## Principal Components Analysis


Perform PCA on the data. Then, plot the trend of

- the singular values $\sigma_k$
- the cumulate fraction of singular values: $\frac{\sum_{i=1}^{k} \sigma_i}{\sum_{i=1}^{q} \sigma_i}$
- the fraction of the "explained variance": $\frac{\sum_{i=1}^{k} \sigma_i^2}{\sum_{i=1}^{q} \sigma_i^2}$


In [ ]:
# SOLUTION-BEGIN
A_mean = A.mean(axis=1)
A_bar = A - A_mean[:, None]
U, s, VT = np.linalg.svd(A_bar, full_matrices=False)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 6))

axes[0].semilogy(s[:-1], "o-")
axes[0].set_title("Singluar values")

axes[1].plot(np.cumsum(s) / np.sum(s), "o-")
axes[1].set_title("Cumulate fraction of singular values")

axes[2].plot(np.cumsum(s**2) / np.sum(s**2), "o-")
axes[2].set_title("Explained variance")
# SOLUTION-END

Make a scatterplot of the first two principal components of the patients.


In [ ]:
# SOLUTION-BEGIN
Phi = U[:, :2].T @ A_bar
plt.scatter(Phi[0, :], Phi[1, :], marker="o", c=(grp == "Normal"), s=50)
plt.show()
# SOLUTION-END

Make a scatterplot of the first three principal components of the patients.


In [ ]:
# SOLUTION-BEGIN
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
Phi = U[:, :3].T @ A_bar
ax.scatter(Phi[0, :], Phi[1, :], Phi[2, :], marker="x", c=(grp == "Normal"), s=50)
ax.view_init(25, 20)
plt.show()
# SOLUTION-END

Or more compactly, and with an interactive plot


In [ ]:
import plotly.express as px

px.scatter_3d(x=Phi[0, :], y=Phi[1, :], z=Phi[2, :], color=grp)

Visualize the protein components of the first three principal components

In [ ]:
# SOLUTION-BEGIN
plt.plot(U[:, 0:3])
plt.legend(["PC1", "PC2", "PC3"])
plt.xlabel("Biomarker ID")
plt.ylabel("Importance")
# SOLUTION-END

The following is a generalization of the thresholding algorithm we have seen in the previous notebook for higher dimensions: instead of a vertical line, think to a plane in 3D that separates the two groups. The implementation of this classification algorithm (SVC) will be an argument of Lab04. For now, we just use this (SVC) as a tool  from a library. We care only about the output training accuracy.

In [ ]:
from sklearn.svm import SVC


def classify(X, labels):
    clf = SVC(kernel="linear")
    clf.fit(X.T, labels)
    y_pred = clf.predict(X.T)
    accuracy = np.mean(y_pred == labels)
    print(f"Training accuracy: {accuracy:.2f}")

classify(Phi, is_sane)